Below are the coordinates of the Sunnyvale city open street map data I will be working on. I have downloadede the data from the Overpass API link provided in the project details. 
Minimum Latitude : 37.3499 Maximum Latitude: 37.3863 Minimum Longitude: -122.0654 Maximum Longitude: -121.9782

Below is the code to understand different tags in the data and counts against each of them.

In [20]:
import xml.etree.cElementTree as ET
import pprint

def count_tags(filename):
    # YOUR CODE HERE
    tags = {}
    osm_file = open(filename, 'r')
    for event, elem in ET.iterparse(osm_file):
        #print elem.tag
        if elem.tag in tags:
            tags[elem.tag] = tags[elem.tag]+1
        else:
            tags[elem.tag] = 1
    return tags

data = 'C:/Users/Saisandeep/Documents/Udacity Data Analyst/Data wrangling/interpreter'
tags = count_tags(data)
pprint.pprint(tags)


{'member': 7369,
 'meta': 1,
 'nd': 293169,
 'node': 250586,
 'note': 1,
 'osm': 1,
 'relation': 499,
 'tag': 96905,
 'way': 35156}


From the above code I can see that there are Nodes and Ways. I will be working on getting these into a database and analyzing further.

In [21]:
import xml.etree.cElementTree as ET
import pprint
import re

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


def key_type(element, keys):
    if element.tag == "tag":
        
        if lower.match(element.attrib['k']):
            keys['lower'] = keys['lower']+1
        elif lower_colon.match(element.attrib['k']):
            keys['lower_colon'] = keys['lower_colon'] + 1
        elif problemchars.search(element.attrib['k']):
            keys['problemchars'] =keys['problemchars'] + 1
        else:
            keys['other'] = keys['other'] +1
        #print element.attrib['k']
        #print keys
    return keys

def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)
    return keys

keys = process_map(data)
pprint.pprint(keys)


{'lower': 63283, 'lower_colon': 32580, 'other': 1042, 'problemchars': 0}


I have used the code I worked on in the exercise. Looks like there are lower colons in 32580 tags. I want to continue the way we have split the keys into type and the key.

In [22]:
import xml.etree.cElementTree as ET
import pprint
import re

def get_user(element):
    return


def process_map(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        if element.tag in ['node', 'way', 'relation']:
            #print element.tag
            users.add(element.attrib['user'])
    return users

users = process_map(data)
pprint.pprint(users)


set(['25or6to4',
     '42429',
     'ACE|Tony',
     'AKOK',
     'Aaron Lidman',
     'Ahlzen',
     'Aidan703',
     'Alan Pierrat',
     'Alan Vogt',
     'Aleks-Berlin',
     'Alexander Avtanski',
     'AndiG88',
     'Andrew Hazlett',
     'AndrewSnow',
     'Ankur Datta',
     'Apo42',
     'Ashish Vijayaram',
     'Ashvini Kumar',
     'Aury88',
     'AvinashR',
     'Azlux',
     'BCNorwich',
     'Ben Luo',
     'Bhaskar U',
     'Bhojaraj',
     'Bike Mapper',
     'Blue Skyes Acupuncture',
     'Bman',
     'BobbyAndThomas',
     'Brian2433',
     'Brian@Brea',
     'Bryce C Nesbitt',
     'BugBuster',
     'BuganiniQ',
     'Cato_d_Ae',
     'Chaitanya Kumar CH',
     'Chetan91',
     'Chetan_Gowda',
     'ChrisZontine',
     'CitymapperHQ',
     'CloCkWeRX',
     'CongoMan',
     'CoreyFarwell',
     'DanHomerick',
     'Dana_Bringas',
     'Daniel_Caldwell',
     'David Muir Sharnoff',
     'David Speakman',
     'Dero Bike Racks',
     'Dhruv Matani',
     'Dr Kludge',
 

From the above code I have got the list of all the users who have contributed in creating this data from nodes, ways and relations.

In [5]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

OSMFILE = data
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", "Terrace", "Way", "Expressway", "Real"]

mapping = { "St": "Street",
            "St.": "Street",
            "Ave": "Avenue",
            "Ave.": "Avenue",
            "Dr": "Drive",
            "Ct": "Court",
            "Pl": "Place",
            "Ln": "Lane",
            "Rd": "Road",
            "Rd.": "Road",
            "PKWY": "Parkway",
            "Blvd": "Boulevard"
            }


def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types


def update_name(name, mapping):
    m = street_type_re.search(name)
    if m:
        street_type = m.group()
        if street_type in mapping:
            name = name[:-len(street_type)] + mapping[street_type]
            #name = name.replace(street_type, mapping[street_type])
    return name


st_types = audit(OSMFILE)
pprint.pprint(dict(st_types))
print "------------------------------------"

for st_type, ways in st_types.iteritems():
    for name in ways:
        better_name = update_name(name, mapping)
        if name != better_name:
            print name, "=>", better_name

{'1': set(['Stewart Drive Suite #1']),
 '114': set(['West Evelyn Avenue Suite #114']),
 'Ave': set(['W Washington Ave']),
 'Circle': set(['Bobolink Circle', 'Continental Circle']),
 'East': set(['Vanderbilt Court East']),
 'Oaks': set(['North Fair Oaks']),
 'Rd': set(['Wolfe Rd']),
 'West': set(['Vanderbilt Court West'])}
------------------------------------
Wolfe Rd => Wolfe Road
W Washington Ave => W Washington Avenue


In [12]:
# Phone numbers analysis
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint
data = 'C:/Users/Saisandeep/Documents/Udacity Data Analyst/Data wrangling/interpreter'
OSMFILE = data

#below function would audit all types of phone numbers. I worked on the regex to build it incrementally adding all the different formats of phone numbers
def auditphone(osmfile):
    osm_file = open(osmfile, "r")
    phone_type = re.compile(r'^1\.[0-9]{3}\.[0-9]{3}\.[0-9]{4}$|^\+1\-[0-9]{3}\-[0-9]{3}\-[0-9]{4}$|^[0-9]{3}\.[0-9]{3}\.[0-9]{4}$|^[0-9]{3}\.[0-9]{3}\.[A-Z]{4}$|^[0-9]{10}$|^\+1\s[0-9]{3}\s[0-9]{3}\s[0-9]{4}$|^1\s[0-9]{3}\s[0-9]{3}\s[0-9]{4}$|^[0-9]{3}\s[0-9]{3}\s[0-9]{4}$|^[0-9]{3}\-[0-9]{3}\-[0-9]{4}$|^\([0-9]{3}\)\s[0-9]{3}\-[0-9]{4}$|^\+1\.[0-9]{3}\.[0-9]{3}\.[0-9]{4}$', re.IGNORECASE)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if tag.attrib['k']=='phone':
                    m = phone_type.search(tag.attrib['v'])
                    if not m:
                        print tag.attrib['v']
    osm_file.close()
     
auditphone(OSMFILE)

#Below function is used to conver the alphabets in the phone numbers to the numbers based on the T9 formating. I refered to a stackoverflow code for this.
def alpha_num(input):
  
    key_alph='abcdefghijklmnopqrstuvwxyz'
    
    key_num= '22233344455566677778889999'
    
    counter = len(input)
    #print counter
    total=''    #Stores the part of string which is not going to be changed
    #print total
    while (counter>0):
        alpha = input.lower()[-counter]
        #print counter
        #print alpha
        if alpha.isalpha():   #checks if each character in the input is a valid alphabet and not a number.
            total+=key_num[key_alph.index(alpha)]  #Appending the new characters  
        else:
            total+=alpha     #This will preserve if any numeric character is encountered and keeps it same as in the input
        counter -= 1
    return total

# below function corrects all different formats of numbers into standart format +1XXXXXXXXXX
def format_phone(test):

    if re.match('^[0-9]{10}$', test):
        number = '+1'+test
    elif re.match('^1[0-9]{10}$',test):
        number = '+'+test
    elif re.match('^\+1[0-9]{10}$',test):
        number = test
    elif re.match('^\([0-9]{3}\)\s[0-9]{3}\-[0-9]{4}$',test):
        number = '+1'+test[1:4]+test[6:9]+test[10:]
    elif re.match('^\([0-9]{3}\)\-[0-9]{3}\-[0-9]{4}$',test):
        number = '+1'+test[1:4]+test[6:9]+test[10:]
    elif re.match('^\+1\s\([0-9]{3}\)\s[0-9]{3}\-[0-9]{4}$',test):
        number = '+1'+test[4:7]+test[9:12]+test[13:]
    elif re.match('^[0-9]{3}\s[0-9]{3}\s[0-9]{4}$',test):
        number = '+1'+test.replace(' ','')
    elif re.match('^\+1\s[0-9]{3}\s[0-9]{3}\s[0-9]{4}$',test):
        number = test.replace(' ','')
    elif re.match('^\+1\s[0-9]{3}\s[0-9]{3}\-[0-9]{4}$',test):
        number = test.replace(' ','').replace('-','')
    elif re.match('^\+1\-[0-9]{3}\-[0-9]{3}\-[0-9]{4}$',test):
        number = test.replace('-','')
    elif re.match('^\+1\s[0-9]{3}\s[0-9]{7}$',test):
        number = test.replace(' ','')
    elif re.match('^\+1\s[0-9]{3}\-[0-9]{3}\-[0-9]{4}$',test):
        number = test.replace(' ','').replace('-','')
    elif re.match('^\+1\.[0-9]{3}\.[0-9]{3}\.[0-9]{4}$',test):
        number = test.replace('.','')
    elif re.match('^1\.[0-9]{3}\.[0-9]{3}\.[0-9]{4}$',test):
        number = '+'+test.replace('.','')
    elif re.match('^1\s[0-9]{3}\s[0-9]{3}\s[0-9]{4}$',test):
        number = '+'+test.replace(' ','')
    elif re.match('^[0-9]{3}\.[0-9]{3}\.[0-9]{4}$',test):
        number = '+1'+test.replace('.','')
    elif re.match('^[0-9]{3}\-[0-9]{3}\-[0-9]{4}$',test):
        number = '+1'+test.replace('-','')
    elif re.match('^1\-[0-9]{3}\-[0-9]{3}\-[0-9]{4}$',test):
        number = test.replace('-','')
    elif re.match('^\+1\-[0-9]{3}\-[0-9]{3}\-[a-z]{4}$',test,flags=re.IGNORECASE):
        number = '+1'+test[2:5]+test[6:9]+alpha_num(test[10:])
    elif re.match('^\([0-9]{3}\)\s[0-9]{3}\-[a-z]{4}$',test,flags=re.IGNORECASE):
        number = '+1'+test[1:4]+test[6:9]+alpha_num(test[10:])
    else:
        number = 'N/A'
    test = number
    return number

#test = '1-650-938-NYNY'
#print test
#print format_phone(test)



(408) 738-CHEF
+1 408 7339000
+1 408 746-0646
+1-650-938-NYNY
+16502898186
+1 408-962-0396
+1 408-730-9200
+1 408-260-2727
+1 408-247-4300
(408)-732-0300
+1 408 2691048
+1 (408) 245-8434


In [13]:
for event, elem in ET.iterparse(OSMFILE, events=("start",)):
    if elem.tag == "node" or elem.tag == "way":
        for tag in elem.iter("tag"):
            if tag.attrib['k'] == 'phone':
                #print 'woohoo'
                print tag.attrib['v']
                print format_phone(tag.attrib['v'])


+1 408 736 6859
+14087366859
4087382177
+14087382177
+1 408 733 1304
+14087331304
408-498-4293
+14084984293
+1 408 739 7717
+14087397717
(408) 730-5510
+14087305510
+1 408 774 1889
+14087741889
+1-408-423-8502
+14084238502
4087369496
+14087369496
(408) 732-1573
+14087321573
4087386825
+14087386825
4087388761
+14087388761
4087361114
+14087361114
4088309933
+14088309933
4087388515
+14087388515
+1.408.245.5300
+14082455300
+1 408 830 9100
+14088309100
+1 408 220 6588
+14082206588
408 245 8988
+14082458988
+1 408 245 4423
+14082454423
+1 408 739 1282
+14087391282
+1 408 737 2852
+14087372852
+1 408 720 8111
+14087208111
+1-408-245-6463
+14082456463
+1-408-732-6218
+14087326218
+1 408 732 4280
+14087324280
+1 408 245 4100
+14082454100
650-969-3382
+16509693382
(408) 733-9446
+14087339446
+1 408 736 9888
+14087369888
+1 408 739 2251
+14087392251
(408) 530-9555
+14085309555
(408) 730-8784
+14087308784
(408) 735-9830
+14087359830
408.773.8095
+14087738095
(408) 738-CHEF
+14087382433
4082458828


From the above code I clearly see that it is lot clean already. There are only 2 places where the mapping I developed in the exercise was useful. Other street names which are listed above look genuine addresses only.

In [24]:
ls

 Volume in drive C is Windows8_OS
 Volume Serial Number is 5298-EB2A

 Directory of C:\Users\Saisandeep\Documents\Udacity Data Analyst\Data wrangling

09/08/2017  01:44 PM    <DIR>          .
09/08/2017  01:44 PM    <DIR>          ..
08/22/2017  05:46 AM    <DIR>          .ipynb_checkpoints
07/30/2017  04:09 PM         1,572,864 2013_ERCOT_Hourly_Load_Data.xls
07/30/2017  11:46 AM           746,002 2013_ERCOT_Hourly_Load_Data.zip
07/30/2017  04:09 PM               353 2013_Max_Loads.csv
08/22/2017  06:48 AM    <DIR>          cerberus-master
08/22/2017  06:47 AM           145,110 cerberus-master.zip
09/08/2017  01:44 PM            54,933 Data Wrangling OSM Project.ipynb
09/06/2017  05:46 AM           390,288 Data+Wrangling+OSM+Project.html
08/12/2017  05:45 PM            37,054 Datawranglingrough.ipynb
08/21/2017  07:29 AM        55,816,388 interpreter
08/28/2017  07:12 PM        21,006,789 nodes.csv
08/22/2017  08:01 AM           511,486 nodes_tags.csv
09/08/2017  06:18 AM    <DIR>    

In [25]:

schema = {
    'node': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'lat': {'required': True, 'type': 'float', 'coerce': float},
            'lon': {'required': True, 'type': 'float', 'coerce': float},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'node_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    },
    'way': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'way_nodes': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'node_id': {'required': True, 'type': 'integer', 'coerce': int},
                'position': {'required': True, 'type': 'integer', 'coerce': int}
            }
        }
    },
    'way_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    }
}


I have worked on the creating the csv files from the xml using the code worked on during exercise. I faced some challenges in making this code work due to multiple reasons. But I figured out how to fix it based on the error messages and print statements at every stage to understand error. 

Initially I could not get the packages Cerberus and Schema installed on my environment as conda was not able to search for them. Later I searched for a while to figure out the correct name it had to install with and some documentation on what these are used for. I enjoyed reading the materail and understand how the complete code is working. 

I also struggled using schema.py but the udacity forums helped me look for a solution for this issue. It is great learning for the furture.

In [14]:

import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET

import cerberus

import schema

data = 'C:/Users/Saisandeep/Documents/Udacity Data Analyst/Data wrangling/interpreter'
OSM_PATH = data

NODES_PATH = "C:/Users/Saisandeep/Documents/Udacity Data Analyst/Data wrangling/nodes.csv"
NODE_TAGS_PATH = "C:/Users/Saisandeep/Documents/Udacity Data Analyst/Data wrangling/nodes_tags.csv"
WAYS_PATH = "C:/Users/Saisandeep/Documents/Udacity Data Analyst/Data wrangling/ways.csv"
WAY_NODES_PATH = "C:/Users/Saisandeep/Documents/Udacity Data Analyst/Data wrangling/ways_nodes.csv"
WAY_TAGS_PATH = "C:/Users/Saisandeep/Documents/Udacity Data Analyst/Data wrangling/ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    #print element.tag
    #print element.attrib
    if element.tag == 'node':
        for field in node_attr_fields:
            #print field, element.attrib[field]
            node_attribs[field] = element.attrib[field]
    if element.tag == 'way':
        for field in way_attr_fields:
            #print field, element.attrib[field]
            way_attribs[field] = element.attrib[field]
        count = 0
        for node in element.findall('nd'):
            entry = {}
            entry['id'] = element.attrib['id']
            entry['node_id'] = node.attrib['ref']
            entry['position'] = count
            count = count +1
            way_nodes.append(entry)
        #print '-------------------------'
        #print way_nodes
                
    for tag in element.findall('tag'):
        entry = {}
        
        if is_street_name(tag):
            entry['value'] = update_name(tag.attrib['v'], mapping)
        elif tag.attrib['k'] == 'phone':
            entry['value'] = format_phone(tag.attrib['v'])
        else:
            entry['value'] = tag.attrib['v']
        
        if ':' in tag.attrib['k']:
            keys = tag.attrib['k'].split(':',1)
            entry['type'] = keys[0]
            entry['key'] = keys[1]
        else:
            entry['type'] = 'regular'
            entry['key'] = tag.attrib['k']
        entry['id'] = element.attrib['id']
        tags.append(entry)
        
        
    #print'-----------------------'
    #print tags
    
    if element.tag == 'node':
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


process_map(OSM_PATH, validate = True)


 I faced challenges loading file into database. Initially when I tried opening the csv files in excel it showed empty alternate rows. so I wrote the below function to confirm if there are empty rows.

In [12]:
import csv
filename = 'C:/Users/Saisandeep/Documents/Udacity Data Analyst/Data wrangling/ways.csv'

def emptyrowcounts(filename):
    count = 0
    rows = 0
    with open(filename, 'r') as f:
        for row in f:
            if row:
                rows = rows + 1
            else:
                count = count+1
    return rows, count

emptyrowcounts(filename)

(35157, 0)

I ran the above code on all the files and found there are no empty rows in the files and continued my struggle with the loading into database. After quite a bit of struggle and search online I realized its the issue with the datatype and decoding. Then I reached out the forums to check if someone else had similar issues and found a lot of help and guidence from previous discussions. I was able to figure out my mistakes and used python connection to sqlite database and performed below queries.

In [18]:
import sqlite3
import csv
import pprint

database = 'C:/sqlite_windows/osm.db'

con = sqlite3.connect(database)
cur = con.cursor()

cur.execute('''DROP TABLE IF EXISTS nodes; ''')
con.commit()

cur.execute("CREATE TABLE nodes (id INTEGER PRIMARY KEY NOT NULL, lat REAL, lon REAL, user TEXT, uid INTEGER, version TEXT, changeset INTEGER, timestamp DATE);") # use your column names here
con.commit()
with open('C:/sqlite_windows/nodes.csv','rb') as fin: # `with` statement available in 2.5+
    # csv.DictReader uses first line in file for column headings by default
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['id'].decode("utf-8"),i['lat'].decode("utf-8"),i['lon'].decode("utf-8"),\
              i['user'].decode("utf-8"),i['uid'].decode("utf-8"),i['version'].decode("utf-8"),\
              i['changeset'].decode("utf-8"), i['timestamp'].decode("utf-8")) for i in dr]


cur.executemany("INSERT INTO nodes (id, lat, lon, user, uid, version, changeset, timestamp) VALUES (?,?,?,?,?,?,?,?);", to_db)
con.commit()
con.close()

In [18]:
import sqlite3
import csv
import pprint
database = 'C:/sqlite_windows/osm.db'

con = sqlite3.connect(database)
cur = con.cursor()


cur.execute('''DROP TABLE IF EXISTS nodes_tags; ''')
con.commit()

cur.execute("CREATE TABLE nodes_tags (\
    id INTEGER,\
    key TEXT,\
    value TEXT,\
    type TEXT,\
    FOREIGN KEY (id) REFERENCES nodes(id)\
);") # use your column names here
con.commit()

with open('C:/sqlite_windows/nodes_tags.csv','rb') as fin: # `with` statement available in 2.5+
    # csv.DictReader uses first line in file for column headings by default
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['id'].decode("utf-8"),i['key'].decode("utf-8"),i['value'].decode("utf-8"),\
              i['type'].decode("utf-8")) for i in dr]


cur.executemany("INSERT INTO nodes_tags (id, key, value, type) VALUES (?,?,?,?);", to_db)
con.commit()
con.close()

In [23]:
database = 'C:/sqlite_windows/osm.db'

con = sqlite3.connect(database)
cur = con.cursor()


cur.execute('''DROP TABLE IF EXISTS ways; ''')
con.commit()

cur.execute("CREATE TABLE ways (id INTEGER PRIMARY KEY NOT NULL, user TEXT,uid INTEGER,version TEXT,changeset INTEGER,timestamp TEXT);") # use your column names here
con.commit()

with open('C:/sqlite_windows/ways.csv','rb') as fin: # `with` statement available in 2.5+
    # csv.DictReader uses first line in file for column headings by default
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['id'].decode("utf-8"),i['user'].decode("utf-8"),i['uid'].decode("utf-8"),\
              i['version'].decode("utf-8"),i['changeset'].decode("utf-8"),i['timestamp'].decode("utf-8")) for i in dr]


cur.executemany("INSERT INTO ways (id, user, uid, version, changeset, timestamp) VALUES (?,?,?,?,?,?);", to_db)
con.commit()
con.close()

In [25]:
database = 'C:/sqlite_windows/osm.db'

con = sqlite3.connect(database)
cur = con.cursor()


cur.execute('''DROP TABLE IF EXISTS ways_nodes; ''')
con.commit()

cur.execute("CREATE TABLE ways_nodes (id INTEGER NOT NULL,node_id INTEGER NOT NULL,position INTEGER NOT NULL,\
FOREIGN KEY (id) REFERENCES ways(id),FOREIGN KEY (node_id) REFERENCES nodes(id));") # use your column names here
con.commit()

with open('C:/sqlite_windows/ways_nodes.csv','rb') as fin: # `with` statement available in 2.5+
    # csv.DictReader uses first line in file for column headings by default
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['id'].decode("utf-8"),i['node_id'].decode("utf-8"),i['position'].decode("utf-8")) for i in dr]


cur.executemany("INSERT INTO ways_nodes (id, node_id, position) VALUES (?,?,?);", to_db)
con.commit()
con.close()

In [19]:
database = 'C:/sqlite_windows/osm.db'

con = sqlite3.connect(database)
cur = con.cursor()


cur.execute('''DROP TABLE IF EXISTS ways_tags; ''')
con.commit()

cur.execute("CREATE TABLE ways_tags (id INTEGER NOT NULL,key TEXT NOT NULL,value TEXT NOT NULL,\
type TEXT,FOREIGN KEY (id) REFERENCES ways(id));") # use your column names here
con.commit()

with open('C:/sqlite_windows/ways_tags.csv','rb') as fin: # `with` statement available in 2.5+
    # csv.DictReader uses first line in file for column headings by default
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['id'].decode("utf-8"),i['key'].decode("utf-8"),i['value'].decode("utf-8"),\
              i['type'].decode("utf-8")) for i in dr]


cur.executemany("INSERT INTO ways_tags (id, key, value, type) VALUES (?,?,?,?);", to_db)
con.commit()
con.close()

In [18]:
from pprint import pprint 
import sqlite3

def sqlquery(query):
    database = 'C:/sqlite_windows/osm.db'
    con = sqlite3.connect(database)
    cur = con.cursor()
    cur.execute(query)
    pprint(cur.fetchall())
    con.close()

query = 'select user, uid, count(*) from nodes group by uid order by count(*) desc limit 10;'
sqlquery(query)

[(u'andygol', 94578, 36924),
 (u'RichRico', 2219338, 31013),
 (u'karitotp', 2748195, 29270),
 (u'samely', 2512300, 24964),
 (u'n76', 318696, 21835),
 (u'dannykath', 2226712, 21037),
 (u'ediyes', 1240849, 17715),
 (u'calfarome', 2511706, 10594),
 (u'matthieun', 595221, 9457),
 (u'nikhilprabhakar', 2835928, 8185)]


In [72]:
query = 'select user, uid, count(*) from nodes group by uid order by count(*) desc limit 10;'
sqlquery(query)

[(u'andygol', 94578, 36924),
 (u'RichRico', 2219338, 31013),
 (u'karitotp', 2748195, 29270),
 (u'samely', 2512300, 24964),
 (u'n76', 318696, 21835),
 (u'dannykath', 2226712, 21037),
 (u'ediyes', 1240849, 17715),
 (u'calfarome', 2511706, 10594),
 (u'matthieun', 595221, 9457),
 (u'nikhilprabhakar', 2835928, 8185)]


In [73]:
query = 'select key, count(*) from nodes_tags group by key order by count(*) desc limit 10;'
sqlquery(query)

[(u'housenumber', 2371),
 (u'street', 2320),
 (u'city', 1516),
 (u'highway', 1458),
 (u'name', 736),
 (u'amenity', 523),
 (u'source', 354),
 (u'postcode', 325),
 (u'addr', 257),
 (u'shop', 251)]


In [74]:
query = 'select value, count(*) from nodes_tags where key = \'amenity\' group by value order by count(*) desc limit 10;'
sqlquery(query)

[(u'restaurant', 127),
 (u'bicycle_parking', 68),
 (u'bench', 34),
 (u'fast_food', 31),
 (u'dentist', 23),
 (u'cafe', 22),
 (u'doctors', 20),
 (u'parking', 17),
 (u'atm', 15),
 (u'post_box', 15)]


In [75]:
query = 'select value, count(*) from nodes_tags where key = \'shop\' group by value order by count(*) desc limit 10;'
sqlquery(query)

[(u'hairdresser', 25),
 (u'car_repair', 21),
 (u'beauty', 18),
 (u'supermarket', 15),
 (u'dry_cleaning', 11),
 (u'convenience', 10),
 (u'alcohol', 9),
 (u'laundry', 8),
 (u'clothes', 7),
 (u'jewelry', 6)]


In [76]:
query = 'select value, count(*) from nodes_tags where key = \'highway\' group by value order by count(*) desc limit 10;'
sqlquery(query)

[(u'stop', 504),
 (u'turning_circle', 346),
 (u'traffic_signals', 260),
 (u'crossing', 256),
 (u'bus_stop', 57),
 (u'give_way', 21),
 (u'motorway_junction', 10),
 (u'turning_loop', 3),
 (u'survey', 1)]


In [77]:
query = 'select id, count(*) from ways_nodes group by id order by count(*) desc limit 10;'
sqlquery(query)

[(250195022, 164),
 (38201877, 143),
 (39200808, 135),
 (49438764, 135),
 (228906414, 126),
 (49438925, 114),
 (390153611, 113),
 (390153610, 112),
 (47795407, 109),
 (474246349, 107)]


In [78]:
    query = 'select id, count(*) from ways_tags group by id order by count(*) desc limit 10;'
    sqlquery(query)

[(49046539, 21),
 (26230997, 20),
 (22372547, 19),
 (26230999, 18),
 (30931570, 18),
 (340316602, 18),
 (41914255, 17),
 (49322007, 17),
 (311341167, 17),
 (311341170, 17)]


In [79]:
query = 'select key, count(*) from ways_tags group by key order by count(*) desc limit 10;'
sqlquery(query)

[(u'building', 27627),
 (u'source', 6558),
 (u'highway', 5849),
 (u'street', 5341),
 (u'housenumber', 5338),
 (u'name', 3203),
 (u'maxspeed', 2614),
 (u'service', 2268),
 (u'lanes', 1964),
 (u'county', 1932)]


In [80]:
query = 'select value, count(*) from ways_tags where key = \'amenity\' group by value order by count(*) desc limit 10;'
sqlquery(query)

[(u'parking', 140),
 (u'restaurant', 26),
 (u'school', 26),
 (u'fuel', 23),
 (u'bank', 18),
 (u'fast_food', 17),
 (u'place_of_worship', 15),
 (u'swimming_pool', 10),
 (u'fountain', 7),
 (u'pharmacy', 5)]


In [81]:
query = 'select a.user, b.value, count(*) from nodes a join nodes_tags b on a.id = b.id \
         where b.key = \'amenity\' group by b.value order by count(*) desc limit 10;'
sqlquery(query)

[(u'Perch338', u'restaurant', 127),
 (u'Harry Cutts', u'bicycle_parking', 68),
 (u'Minh Nguyen', u'bench', 34),
 (u'jgkamat', u'fast_food', 31),
 (u'jgkamat', u'dentist', 23),
 (u'WrErase', u'cafe', 22),
 (u'nmixter', u'doctors', 20),
 (u'Jonathan ZHAO', u'parking', 17),
 (u'matthieun', u'atm', 15),
 (u'fmarier', u'post_box', 15)]


In [82]:
query = 'select key, count(*) from nodes_tags group by key order by count(*) desc limit 20;'
sqlquery(query)

[(u'housenumber', 2371),
 (u'street', 2320),
 (u'city', 1516),
 (u'highway', 1458),
 (u'name', 736),
 (u'amenity', 523),
 (u'source', 354),
 (u'postcode', 325),
 (u'addr', 257),
 (u'shop', 251),
 (u'crossing', 240),
 (u'direction', 240),
 (u'country', 193),
 (u'state', 181),
 (u'phone', 171),
 (u'cuisine', 154),
 (u'created_by', 133),
 (u'surface', 130),
 (u'website', 129),
 (u'date', 128)]


In [83]:
query = 'select a.user, b.value, count(*) from nodes a join nodes_tags b on a.id = b.id \
         where b.key = \'cuisine\' group by b.value order by count(*) desc limit 10;'
sqlquery(query)

[(u'Perch338', u'indian', 25),
 (u'mvexel', u'chinese', 15),
 (u'jgkamat', u'sandwich', 15),
 (u'joeclmbr', u'pizza', 14),
 (u'n76', u'mexican', 9),
 (u'n76', u'thai', 8),
 (u'Minh Nguyen', u'korean', 7),
 (u'Minh Nguyen', u'burger', 5),
 (u'WrErase', u'coffee_shop', 5),
 (u'Minh Nguyen', u'japanese', 5)]


In [3]:
query = 'SELECT tags.value, COUNT(*) as count FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) tags \
WHERE tags.key= \'postcode\' GROUP BY tags.value ORDER BY count DESC;'
sqlquery(query)

[(u'94087', 250),
 (u'94086', 231),
 (u'95051', 137),
 (u'94085', 25),
 (u'94040', 18),
 (u'94086-6406', 5),
 (u'95054', 5),
 (u'94041', 3),
 (u'94807', 2),
 (u'94087-2248', 1),
 (u'94087\u200e', 1),
 (u'94088-3453', 1),
 (u'94088-3707', 1),
 (u'95050', 1),
 (u'95086', 1),
 (u'CA 94086', 1)]


In [4]:
query = 'SELECT COUNT(*) FROM nodes;'
sqlquery(query)

[(250586,)]


In [5]:
query = 'SELECT COUNT(*) FROM ways;'
sqlquery(query)

[(35156,)]


In [6]:
query = 'SELECT COUNT(DISTINCT(e.uid)) FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) e'
sqlquery(query)

[(324,)]


In [19]:
query = 'SELECT e.user, COUNT(*) as num \
FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e \
GROUP BY e.user \
ORDER BY num DESC \
LIMIT 10;' 
sqlquery(query)

[(u'andygol', 41907),
 (u'RichRico', 34222),
 (u'karitotp', 31478),
 (u'samely', 28559),
 (u'n76', 27198),
 (u'dannykath', 23103),
 (u'ediyes', 19664),
 (u'calfarome', 11449),
 (u'matthieun', 11309),
 (u'nikhilprabhakar', 9047)]


In [8]:
query = 'SELECT COUNT(*) FROM (SELECT e.user, COUNT(*) as num FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e\
 GROUP BY e.user HAVING num=1)  u;'
sqlquery(query)

[(85,)]


In [12]:
query = 'SELECT nodes_tags.value, COUNT(*) as num FROM nodes_tags JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value=\'place_of_worship\') i \
ON nodes_tags.id=i.id WHERE nodes_tags.key=\'religion\' GROUP BY nodes_tags.value ORDER BY num DESC;'
sqlquery(query)

[(u'christian', 11), (u'unitarian_universalist', 1)]


Considering my laptop configuration I choose to restrict myself with the size minimum requirement. 

File sizes:

Sunnyvale.osm --------53.2 MB

sunnyvale.db ---------28.5 MB

nodes.csv ------------20.0 MB

nodes_tags.csv--------500 KB

ways.csv -------------2.0 MB

ways_tags.csv---------2.73 MB

ways_nodes.csv--------6.98 MB

It looks like the Sunnyvale area is incomplete, though I believe it has been well cleaned by the users before I started cleaning itself. I feel lot of information can be added to this data. I live in Sunnyvale and I observed few of the famous restaurants are missing and I was able to search for my home address and it was available in there. I tried to analyze what ever information I could imagine we can get from this data.